In [4]:
%load_ext autoreload
%autoreload 2
%env ANYWIDGET_HMR=1

import numpy as np


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
env: ANYWIDGET_HMR=1


In [7]:
x = np.random.randn(100)
y = np.sin(x)
c = x
s = np.zeros_like(x) + 10 

In [15]:
from dimbridge import Dimbridge

bridge = Dimbridge(x=x, y=y, c=c, s=s)
bridge

Dimbridge(c=array([ 6.58392312e-01, -6.23585038e-01,  1.67137167e+00,  1.25444748e+00,
       -1.06989677e+00,…